In [1]:
import pandas as pd

# Load the CSV file
csv_path = '/Users/guptatilak/Documents/C4GT-Face-Recognition/Dataset/An Indian facial database highlighting the Spectacle 2/Version 2/test_combinations_without_specs.csv'
data = pd.read_csv(csv_path)

data=data.sample(500)


In [2]:
tmp_data=data

In [3]:
data=tmp_data

In [4]:
tmp_data

,img1_path,img2_path,truth_value
2147,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,1
3664,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,0
2186,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,1
302,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,1
3685,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,0
...,...,...,...
4328,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,0
4480,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,0
1944,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,1
1176,/Users/guptatilak/Documents/C4GT-Face-Recognit...,/Users/guptatilak/Documents/C4GT-Face-Recognit...,1


In [5]:
import os
import pandas as pd
from tqdm import tqdm
import cv2
import time
import numpy as np

In [20]:
# example_usage.py

from compare_images import compare_images

# Define image paths
image1_path = 'images/Aisvarrya_8.jpeg'
image2_path = 'images/Rajath_14.jpg'

try:
    # Call the compare_images function from the imported module
    similarity = compare_images(image1_path, image2_path)
    print(f"Cosine Similarity: {similarity:.4f}")
except ValueError as e:
    print(e)


/Users/guptatilak/Documents/C4GT-Face-Recognition/offline-FR/faceboxes-edgeface-FR/embeddings.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torc

Loading pretrained model from weights/FaceBoxes.pth
remove prefix 'module.'
Missing keys:27
Unused checkpoint keys:0
Used keys:147
Loading pretrained model from weights/FaceBoxes.pth
remove prefix 'module.'
Missing keys:27
Unused checkpoint keys:0
Used keys:147
Cosine Similarity: 0.6470


In [ ]:
# Function to check illumination in the image
def check_illumination(filename):
    im = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    if im is None:
        print(f'ERROR: Unable to load {filename}')
    # Calculate mean brightness as percentage
    meanpercent = np.mean(im) * 100 / 255

    if meanpercent > 38:
        return True
    else:
        return False


# Function to denoise an image using Gaussian Blur
def denoise_image(image_path):
    try:
        # Read the image using OpenCV
        img = cv2.imread(image_path)
        if img is not None:
            # Apply Gaussian Blur to denoise the image
            denoised_img = cv2.GaussianBlur(img, (3, 3), 0.5)
            return denoised_img
        else:
            print(f"Error reading image: {image_path}")
            return None
    except (IOError, SyntaxError, ValueError) as e:
        print(f"Error processing image: {image_path} ({e})")
        return None  # You can log the error for further investigation


detector_backend = 'faceboxes'
model_name = 'edgeface'
distance_metric = 'cosine'
align = False

alignment_text = "aligned" if align is True else "unaligned"
task = f"{model_name}_{detector_backend}_{distance_metric}_{alignment_text}"
output_file = f"threshold/outputs/{task}.csv"

distances = []
truth_values = []
times = []
time_gaussian = []

# Test for the first 10 rows of the dataframe only
for index, row in tqdm(data.head(500).iterrows(), total=500, desc=task):
    img1_target = row['img1_path']
    img2_target = row['img2_path']
    truth_value = row['truth_value']

    if check_illumination(img1_target) and check_illumination(img2_target):
        start_gaussian_time = time.time()
        # Denoise images before verification
        denoised_img1 = denoise_image(img1_target)
        denoised_img2 = denoise_image(img2_target)
        end_gaussian_time = time.time()

        if denoised_img1 is None or denoised_img2 is None:
            continue  # Skip this pair if any image couldn't be processed

        start_time = time.time()

        # result = DeepFace.verify(
        #     img1_path=denoised_img1,
        #     img2_path=denoised_img2,
        #     model_name=model_name,
        #     detector_backend=detector_backend,
        #     distance_metric=distance_metric,
        #     align=align,
        #     enforce_detection=False,
        #     # expand_percentage=expand_percentage, # Uncomment and set this if needed
        # )

        result = compare_images(img1_target, img2_target)

        end_time = time.time()

        distance = result
        distances.append(distance)
        truth_values.append(truth_value)
        times.append(end_time - start_time)  # Calculate the time taken and store it
        time_gaussian.append(end_gaussian_time - start_gaussian_time) # Calculate the time taken and store it

    # Calculate the average time taken per pair of images
    avg_time_per_pair = sum(times) / len(times)
    avg_time_per_pair_gaussian = sum(time_gaussian) / len(time_gaussian)
    print(f"Average time taken per pair of images for {task}: {avg_time_per_pair:.4f} seconds")
    print(f"Average time taken per pair of images for Gaussian Blur: {avg_time_per_pair_gaussian:.4f} seconds")


    # -----------------------------------
    df = pd.DataFrame(data.head(10)['truth_value'], columns=["actuals"])
    df['actuals'] = truth_values
    df["distances"] = distances
    df.to_csv(output_file, index=False)

In [11]:
# data = [[0 for _ in range(len(models))] for _ in range(len(detectors))]
# base_df = pd.DataFrame(data, columns=models, index=detectors)

In [1]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score

# Add a dictionary to store thresholds
best_thresholds = {}

result=pd.DataFrame(columns=['model_name','detector_backend','distance_metric','alignment','threshold','accuracy'])

configs = [
    {"detector_backend": "faceboxes", "model_name": "edgeface", "distance_metric": "cosine", "align": False},
]

for config in configs:
    detector_backend = config["detector_backend"]
    model_name = config["model_name"]
    distance_metric = config["distance_metric"]
    is_aligned = config["align"]

    current_df = base_df.copy()
    
    align = "aligned" if is_aligned else "unaligned"
    if detector_backend == "skip" and is_aligned:
        align = "unaligned"

    source_file = f"/Users/guptatilak/Documents/C4GT-Face-Recognition/offline-FR/faceboxes-edgeface-FR/threshold/outputs/{model_name}_{detector_backend}_{distance_metric}_{align}.csv"
    
    if not os.path.exists(source_file):
        print(f"Source file {source_file} does not exist. Skipping...")
        continue

    df = pd.read_csv(source_file)
    
    positive_mean = df[(df["actuals"] == True) | (df["actuals"] == 1)]["distances"].mean()
    negative_mean = df[(df["actuals"] == False) | (df["actuals"] == 0)]["distances"].mean()

    distances = sorted(df["distances"].values.tolist())

    items = []
    for i, distance in enumerate(distances):
        if distance >= positive_mean and distance <= negative_mean:
            sandbox_df = df.copy()
            sandbox_df["predictions"] = False
            idx = sandbox_df[sandbox_df["distances"] < distance].index
            sandbox_df.loc[idx, "predictions"] = True

            actuals = sandbox_df.actuals.values.tolist()
            predictions = sandbox_df.predictions.values.tolist()
            accuracy = 100 * accuracy_score(actuals, predictions)
            items.append((distance, accuracy))

    if items:
        pivot_df = pd.DataFrame(items, columns=["distance", "accuracy"])
        pivot_df = pivot_df.sort_values(by=["accuracy"], ascending=False)
        threshold = pivot_df.iloc[0]["distance"]
        accuracy = pivot_df.iloc[0]["accuracy"]

        # Save the best threshold for the model combination
        best_thresholds[(model_name, detector_backend)] = threshold

        # Add the accuracy to the dataframe
        current_df.at[detector_backend, model_name] = accuracy
        print(f"Best threshold for {model_name} with {detector_backend} and alignment {'TRUE' if align == 'aligned' else 'FALSE'} is {threshold} with accuracy {accuracy}")
        data_to_append = {'model_name': model_name,
                  'detector_backend': detector_backend,
                  'distance_metric':distance_metric,
                  'alignment' : 'TRUE' if align == 'aligned' else 'FALSE',
                  'threshold': threshold,
                  'accuracy': accuracy}

        result = result.append(data_to_append, ignore_index=True)
    else:
        print(f"No valid distances for {model_name} with {detector_backend}")


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#using custom thresholds
import os
import pandas as pd
from sklearn.metrics import accuracy_score

# Dictionary to store the custom thresholds for each model combination
custom_thresholds = {
    ("Dlib", "retinaface"): 0.06,
}

result = pd.DataFrame(columns=['model_name', 'detector_backend', 'distance_metric', 'alignment', 'threshold', 'accuracy'])

configs = [
    {"detector_backend": "yolov8", "model_name": "Dlib", "distance_metric": "cosine", "align": True},
    {"detector_backend": "retinaface", "model_name": "Dlib", "distance_metric": "cosine", "align": True},
    {"detector_backend": "yolov8", "model_name": "GhostFaceNet", "distance_metric": "cosine", "align": True},
    {"detector_backend": "retinaface", "model_name": "GhostFaceNet", "distance_metric": "cosine", "align": True}
]

for config in configs:
    detector_backend = config["detector_backend"]
    model_name = config["model_name"]
    distance_metric = config["distance_metric"]
    is_aligned = config["align"]

    align = "aligned" if is_aligned else "unaligned"
    if detector_backend == "skip" and is_aligned:
        align = "unaligned"

    source_file = f"/Users/guptatilak/Documents/C4GT-Face-Recognition/Testing/outputs/test/{model_name}_{detector_backend}_{distance_metric}_{align}.csv"
    # source_file = '/Users/guptatilak/Documents/C4GT-Face-Recognition/Dlib_retinaface_cosine_aligned.csv'

    if not os.path.exists(source_file):
        print(f"Source file {source_file} does not exist. Skipping...")
        continue

    df = pd.read_csv(source_file)
    
    # Check if a custom threshold exists for the current model configuration
    if (model_name, detector_backend) not in custom_thresholds:
        print(f"No custom threshold for {model_name} with {detector_backend}. Skipping...")
        continue
    
    threshold = custom_thresholds[(model_name, detector_backend)]

    sandbox_df = df.copy()
    sandbox_df["predictions"] = False
    idx = sandbox_df[sandbox_df["distances"] < threshold].index
    sandbox_df.loc[idx, "predictions"] = True

    actuals = sandbox_df.actuals.values.tolist()
    predictions = sandbox_df.predictions.values.tolist()
    accuracy = 100 * accuracy_score(actuals, predictions)

    # Add the accuracy to the result dataframe
    data_to_append = {
        'model_name': model_name,
        'detector_backend': detector_backend,
        'distance_metric': distance_metric,
        'alignment': 'TRUE' if align == 'aligned' else 'FALSE',
        'threshold': threshold,
        'accuracy': accuracy
    }

    result = result.append(data_to_append, ignore_index=True)
    print(f"Threshold for {model_name} with {detector_backend} and alignment {'TRUE' if align == 'aligned' else 'FALSE'} is {threshold} with accuracy {accuracy}")

print(result)


Source file /Users/guptatilak/Documents/C4GT-Face-Recognition/Testing/outputs/test/Dlib_yolov8_cosine_aligned.csv does not exist. Skipping...
Threshold for Dlib with retinaface and alignment TRUE is 0.06 with accuracy 94.86458865610629
Source file /Users/guptatilak/Documents/C4GT-Face-Recognition/Testing/outputs/test/GhostFaceNet_yolov8_cosine_aligned.csv does not exist. Skipping...
Source file /Users/guptatilak/Documents/C4GT-Face-Recognition/Testing/outputs/test/GhostFaceNet_retinaface_cosine_aligned.csv does not exist. Skipping...
  model_name detector_backend distance_metric alignment threshold   accuracy
0       Dlib       retinaface          cosine      TRUE      0.06  94.864589


In [ ]:
result

,model_name,detector_backend,distance_metric,alignment,threshold,accuracy
0,Dlib,retinaface,cosine,TRUE,0.06,94.864589


In [ ]:
df=pd.read_csv('/Users/guptatilak/Documents/C4GT-Face-Recognition/offline-FR/faceboxes-edgeface-FR/threshold/outputs/edgeface_faceboxes_cosine_unaligned.csv')

In [ ]:
df

,actuals,distances
0,1,0.989668
1,0,0.942854
2,1,0.877299
3,0,0.671457
4,0,0.819048
...,...,...
392,1,0.991342
393,1,0.341092
394,1,0.996671
395,1,0.658942


In [ ]:
tp_mean = round(df[df.actuals == 1].mean().values[1], 4)
tp_std = round(df[df.actuals == 1].std().values[1], 4)
fp_mean = round(df[df.actuals == 0].mean().values[1], 4)
fp_std = round(df[df.actuals == 0].std().values[1], 4)

In [ ]:
df[df.actuals == 1].distances.plot.kde()
df[df.actuals == 0].distances.plot.kde()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [ ]:
#Statistical Approach
sigma = 2
threshold = round(tp_mean + sigma * tp_std, 4)
threshold

1.2287

In [ ]:
# Import the Chefboost library
from chefboost import Chefboost as chef

# Configuration for the C4.5 algorithm
config = {'algorithm': 'C4.5'}

# Print the column names of the DataFrame to verify
print(df.columns)

# Ensure the column names are correct
if 'actuals' in df.columns and 'distances' in df.columns:
    # Preparing the data
    tmp_df = df[['actuals', 'distances']].rename(columns={"actuals": "Decision"}).copy()
    tmp_df['Decision'] = tmp_df['Decision'].astype(str)  # Convert decision column to string type
    
    # Fit the model
    model = chef.fit(tmp_df, config)

    # After fitting, Chefboost will output the decision tree.
    # You need to manually inspect the printed tree to find the threshold.
else:
    print("Error: The DataFrame does not contain the required columns 'actuals' and 'distances'.")


ModuleNotFoundError: No module named 'chefboost'